In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dataclasses import dataclass, field

from typing import Dict, List, Tuple, Optional, Callable, Any

import math
import numpy as np
from PIL import Image

from pysolotools.consumers import Solo
from pysolotools.converters.solo2coco import SOLO2COCOConverter
from pysolotools.core.models import KeypointAnnotationDefinition, RGBCameraCapture
from pysolotools.core.models import BoundingBox2DLabel, BoundingBox2DAnnotation
from pysolotools.core.models import BoundingBox3DLabel, BoundingBox3DAnnotation
from pysolotools.core.models import Frame, Capture
from scipy.spatial.transform import Rotation as R

from torchvision.datasets import ImageFolder
from torchvision.models import swin_v2_t, Swin_V2_T_Weights
from torchvision.models import swin_v2_b, Swin_V2_B_Weights
# from torch.utils.data import ConcatDataset, DataLoader
from collections import OrderedDict
from torch.utils.data import DataLoader

import torch
from torch import nn, Tensor

from torch.nn import functional as F

from torchvision.ops import FeaturePyramidNetwork, MLP, sigmoid_focal_loss
import lightning.pytorch as pl
from lightning.pytorch.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import LearningRateMonitor

import matplotlib.pyplot as plt
from scipy.optimize import linear_sum_assignment
from scipy.io import savemat

import reader
import utils
import network
import transformer
import cvmpca
from torchvision.ops import FeaturePyramidNetwork
from my_trainer import SetCriterion
torch.set_float32_matmul_precision('medium')
# %matplotlib ipympl

c:\Users\LeonardNgo\miniconda3\envs\cvmpa\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3527.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [3]:
train_folder = 'D:/Unity/dataset/solo'
# training_dir = './data/train'
# testing_dir = './data/test'

In [4]:
train_loader = reader.UnityDataset.from_unity_to_loader(root=train_folder, batch_size=4)

for batch in train_loader:
    image_dicts, object_list = batch

    # for key, image in image_dicts.items():
    #     print(key, image.shape)
    # print('-'*30)
    # for targets in object_list:
    print(object_list.position.shape)

    break

torch.Size([4, 10, 3])


In [5]:
swin = transformer.Swin(is_trainable=True)
fpn = FeaturePyramidNetwork(swin.embed_dims, swin.embed_dim)
embed_dim = swin.embed_dim
embed_dims = swin.embed_dims

In [7]:
# process the image in each view

keys = list(image_dicts.keys())  # camera ids
B = image_dicts[keys[0]].size(0)

visual_features = {
    camera_key: fpn({
        f'feat{i}': x.permute(0, 3, 1, 2)
        for i, x in enumerate(swin(images))
    })
    for camera_key, images in image_dicts.items()
}
for camera_key, features in visual_features.items():
    print(camera_key)
    for key, feature in features.items():
        print(key, feature.shape)
        # break

camera_0
feat0 torch.Size([4, 96, 64, 64])
feat1 torch.Size([4, 96, 32, 32])
feat2 torch.Size([4, 96, 16, 16])
feat3 torch.Size([4, 96, 8, 8])
camera_2
feat0 torch.Size([4, 96, 64, 64])
feat1 torch.Size([4, 96, 32, 32])
feat2 torch.Size([4, 96, 16, 16])
feat3 torch.Size([4, 96, 8, 8])
camera
feat0 torch.Size([4, 96, 64, 64])
feat1 torch.Size([4, 96, 32, 32])
feat2 torch.Size([4, 96, 16, 16])
feat3 torch.Size([4, 96, 8, 8])
camera_1
feat0 torch.Size([4, 96, 64, 64])
feat1 torch.Size([4, 96, 32, 32])
feat2 torch.Size([4, 96, 16, 16])
feat3 torch.Size([4, 96, 8, 8])
